In [2]:
import pandas as pd
import numpy as np
import nltk
import tqdm

from gensim import corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

import tomotopy as tp

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/franciscorfafonso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/franciscorfafonso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Load the DataFrame from the pickle file
df = pd.read_pickle('data_preprocessed_filtered.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57790 entries, 0 to 58446
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      57790 non-null  object 
 1   text                          57790 non-null  object 
 2   created_at                    57790 non-null  object 
 3   campaign_week                 57790 non-null  int64  
 4   process_text_check            57790 non-null  object 
 5   name                          57790 non-null  object 
 6   handle                        57790 non-null  object 
 7   party                         57790 non-null  object 
 8   state_code                    57790 non-null  object 
 9   state_name                    57790 non-null  object 
 10  result_pctg                   57790 non-null  float64
 11  result_votes                  57790 non-null  int64  
 12  position                      57790 non-null  int64  
 13  t

In [4]:
# Function to create text from words
def recreateText(words):
    if type(words) == list:
        temp_str = (' ').join(words)
        return temp_str
    else:
        return np.nan

In [5]:
df['WordsCTM'] = df['WordsLemmatized'].apply(recreateText)
df.head(10)

,tweet_id,text,created_at,campaign_week,process_text_check,name,handle,party,state_code,state_name,...,result_votes,position,total_votes_casted,winner_margin_for_runners_up,check_city_filtered,PreProcessedText_WithoutCity,Words,WordsCleaned,WordsLemmatized,WordsCTM
0,1589769441016877056,Deeply honored to be standing in the historic ...,2022-11-07,36,deeply honored to be standing in the historic ...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,"[exeter, new hampshire]",deeply honored to be standing in the historic ...,"[deeply, honored, to, be, standing, in, the, h...","[deeply, honored, standing, historic, town, ha...","[deeply, honored, standing, historic, town, ha...",deeply honored standing historic town hall ele...
1,1589758870242500608,Full house tonight! Looking forward to having ...,2022-11-07,36,full house tonight looking forward to having w...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,"[exeter, new hampshire]",full house tonight looking forward to having w...,"[full, house, tonight, looking, forward, to, h...","[full, house, tonight, looking, forward, grani...","[full, house, tonight, looking, forward, grani...",full house tonight looking forward granite sta...
2,1589753955592331264,Everyone’s here for the final town hall in Exe...,2022-11-07,36,everyone s here for the final town hall in exe...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,[exeter],everyone s here for the final town hall in be...,"[everyone, s, here, for, the, final, town, hal...","[everyone, final, town, hall, election, many, ...","[everyone, final, town, hall, election, many, ...",everyone final town hall election many came ma...
3,1589723041205886976,Career politicians like @SenatorHassan will sa...,2022-11-07,36,career politicians like will say anything to g...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,NaN,career politicians like will say anything to g...,"[career, politicians, like, will, say, anythin...","[career, politicians, like, say, anything, get...","[career, politician, like, say, anything, get,...",career politician like say anything get reelec...
4,1589684478540406784,@sixohfree Thank you!!! 🇺🇸🇺🇸,2022-11-07,36,thank you,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,NaN,thank you,"[thank, you]",[thank],[thank],thank
5,1589682964933197824,LET'S GO! https://t.co/dpCB6RFCZC,2022-11-07,36,let's go,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,NaN,let's go,"[let's, go]","[let's, go]","[let's, go]",let's go
6,1589682136436465664,🚨🚨🚨Join @GovChrisSununu in supporting our Repu...,2022-11-07,36,join in supporting our republican team in new ...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,"[new hampshire, washington]",join in supporting our republican team in and...,"[join, in, supporting, our, republican, team, ...","[join, supporting, republican, team, #votebold...","[join, supporting, republican, team, #votebold...",join supporting republican team #votebolduc to...
7,1589673963508363265,🚨—ENDORSEMENT ALERT: Thank you to the @LogCabi...,2022-11-07,36,endorsement alert thank you to the for your su...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,NaN,endorsement alert thank you to the for your su...,"[endorsement, alert, thank, you, to, the, for,...","[endorsement, alert, thank, support, campaign,...","[endorsement, alert, thank, support, campaign,...",endorsement alert thank support campaign elect...
8,1589668789863342080,.@newsmax @Tom_Basile covering our Seabrook me...,2022-11-07,36,covering our seabrook meet and greet on the ev...,Don Bolduc,GenDonBolduc,Rep,NH,New Hampshire,...,275928,2,620975,0.090608,"[seabrook, new hampshire, washington]",covering our meet and greet on the eve of ele...,"[covering, our, meet, and, greet, on, the, eve...","[covering, meet, greet, eve, election, day, da...","[covering, meet, gre

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57790 entries, 0 to 58446
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      57790 non-null  object 
 1   text                          57790 non-null  object 
 2   created_at                    57790 non-null  object 
 3   campaign_week                 57790 non-null  int64  
 4   process_text_check            57790 non-null  object 
 5   name                          57790 non-null  object 
 6   handle                        57790 non-null  object 
 7   party                         57790 non-null  object 
 8   state_code                    57790 non-null  object 
 9   state_name                    57790 non-null  object 
 10  result_pctg                   57790 non-null  float64
 11  result_votes                  57790 non-null  int64  
 12  position                      57790 non-null  int64  
 13  t

## Analysis

In [7]:
import os
import datetime

# Create tomotopy_corpus
tomotopy_corpus = tp.utils.Corpus()
print("Creating tomotopy corpus...")

# Add documents to the corpus using a for loop and print the progress
for i, doc in enumerate(df['WordsCTM'], start=1):
    tomotopy_corpus.add_doc(doc.split())
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'{current_time} | {i} of {len(df["WordsCTM"])} total documents added')


print("Created tomotopy corpus...")

Creating tomotopy corpus...
2023-05-09 11:35:46 | 1 of 57790 total documents added
2023-05-09 11:35:46 | 2 of 57790 total documents added
2023-05-09 11:35:46 | 3 of 57790 total documents added
2023-05-09 11:35:46 | 4 of 57790 total documents added
2023-05-09 11:35:46 | 5 of 57790 total documents added
2023-05-09 11:35:46 | 6 of 57790 total documents added
2023-05-09 11:35:46 | 7 of 57790 total documents added
2023-05-09 11:35:46 | 8 of 57790 total documents added
2023-05-09 11:35:46 | 9 of 57790 total documents added
2023-05-09 11:35:46 | 10 of 57790 total documents added
2023-05-09 11:35:46 | 11 of 57790 total documents added
2023-05-09 11:35:46 | 12 of 57790 total documents added
2023-05-09 11:35:46 | 13 of 57790 total documents added
2023-05-09 11:35:46 | 14 of 57790 total documents added
2023-05-09 11:35:46 | 15 of 57790 total documents added
2023-05-09 11:35:46 | 16 of 57790 total documents added
2023-05-09 11:35:46 | 17 of 57790 total documents added
2023-05-09 11:35:46 | 18 of 5

In [8]:

# Create the CTM model
model = tp.CTModel(k=13, min_cf=5, rm_top=10, corpus=tomotopy_corpus)
print("CTM model created.")

# Train the model
print("Training CTM model...")
for i in range(0, 100, 10):
    model.train(10)  # Train the model
    print(f'Iteration: {i}\tLog-likelihood: {model.ll_per_word}')

# Get the topic words for CTM
print("Topics for the CTM model:")
for i in range(model.k):
    topic_words = model.get_topic_words(i, top_n=10)
    words = [word for word, prob in topic_words]
    print('Topic #{}: {}'.format(i, ' '.join(words)))


CTM model created.
Training CTM model...


KeyboardInterrupt: 

In [ ]:
# Create corpus
texts = df['WordsLemmatized']
# Create a Dictionary and a Corpus for Gensim
dictionary = corpora.Dictionary(texts)
# Create TDM (Frequency)
corpus = [dictionary.doc2bow(text) for text in texts]

# Get the topics in a format compatible with Gensim
ctm_topics = [[word for word, _ in model.get_topic_words(i, top_n=10)] for i in range(model.k)]
#lda_topics = [[word for word, _ in lda_model.get_topic_words(i, top_n=10)] for i in range(lda_model.k)]

# Calculate coherence scores using Gensim
ctm_coherence_model = CoherenceModel(topics=ctm_topics, texts=texts, dictionary=dictionary, coherence='c_v')
ctm_coherence = ctm_coherence_model.get_coherence()

#lda_coherence_model = CoherenceModel(topics=lda_topics, texts=cleaned_tweets, dictionary=dictionary, coherence='c_v')
#lda_coherence = lda_coherence_model.get_coherence()

# Print the metrics
print("CTM Coherence: {}".format(ctm_coherence))
#print("LDA Perplexity: {}\tLDA Coherence: {}".format(lda_perplexity, lda_coherence))

# Compare the metrics
#if ctm_coherence > lda_coherence:
#    print("CTM outperforms LDA on both perplexity and coherence.")
#else:
#    print("The models have mixed performance, consider other factors for model selection.")


***Hypertune***

In [ ]:
# Topics range
min_topics = 5
max_topics = 16
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

model_results = {'Topics': [],
                 'Coherence': []
                 }

# Function to compute coherence values for given parameters
def compute_coherence_values(params):
    k, corpus, dictionary, texts = params
    model = tp.CTModel(k=k, min_cf=5, rm_top=10, corpus=corpus)

    print(f"Training CTM model with k={k}...")
    model.train(500)

    ctm_topics = [[word for word, _ in model.get_topic_words(i, top_n=10)] for i in range(model.k)]
    coherence_model = CoherenceModel(topics=ctm_topics, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()

    print(f"Coherence for k={k}: {coherence}")
    return k, coherence

# Can take a long time to run
pbar = tqdm.tqdm(total=len(topics_range))

# Prepare the parameter combinations
params_list = [(k, tomotopy_corpus, dictionary, texts) for k in topics_range]

# Start hyperparameter tuning without parallelization
print("Starting hyperparameter tuning...")
for params in params_list:
    k, cv = compute_coherence_values(params)

    # Save the model results
    model_results['Topics'].append(k)
    model_results['Coherence'].append(cv)

    pbar.update(1)

pd.DataFrame(model_results).to_csv('ctm_tuning_results.csv', index=False)
print("Hyperparameter tuning completed and results saved to 'ctm_tuning_results.csv'")
pbar.close()
